In [1]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from rdkit.Chem import AllChem, Descriptors, MolFromSmiles
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.preprocessing import StandardScaler
import tensorflow as tf

In [4]:
# Import main data and get list of SMILES
freesolv = pd.read_csv('../CRC_Freesolv_addn.csv', index_col = 0)  # Load the freesolv dataset using pandas
smiles_list = freesolv['smiles'].to_list()

In [5]:
freesolv.head()

,Name,Mol. wt.,smiles,kcal/mol,ΔhydH∞/kJ mol-1
0,Acenaphthene,154.207,C1Cc2cccc3cccc1c23,-12.452213,-52.1
1,Acetic acid,60.052,CC(O)=O,-12.619517,-52.8
2,Acetone,58.079,CC(C)=O,-9.488538,-39.7
3,Acetonitrile,41.052,CC#N,-8.341309,-34.9
4,Acetophenone,120.149,CC(=O)c1ccccc1,-12.739020,-53.3


In [6]:
len(smiles_list)

352

In [7]:
# Initiate list of rdkit molecules
rdkit_mols = [MolFromSmiles(smiles) for smiles in smiles_list]

[15:14:59] WARNING: not removing hydrogen atom without neighbors


In [6]:
# Get Morgan fingerprints, note the parameters!
morgan_fingerprints = [AllChem.GetMorganFingerprintAsBitVect(mol, radius=3, nBits=2048) for mol in rdkit_mols]
morgan_fingerprints = np.asarray(morgan_fingerprints)

In [7]:
# Turn into pandas dataframe and add smiles as a first column
morgan_fingerprints = pd.DataFrame(data = morgan_fingerprints)
morgan_fingerprints.insert(0, "smiles", smiles_list)

In [8]:
morgan_fingerprints

,smiles,0,1,2,3,4,5,6,7,8,...,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
0,CN(C)C(=O)c1ccc(cc1)OC,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,CS(=O)(=O)Cl,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,CC(C)C=C,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,CCc1cnccn1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,CCCCCCCO,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
637,CCCCCCCC(=O)OC,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
638,C1CCNC1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
639,c1cc(ccc1C=O)O,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
640,CCCCCCCCl,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
morgan_fingerprints.to_csv("freesolv_morgan_fingerprints.csv")

In [8]:
# Next, rdkit's own descriptors
from rdkit.Chem import Descriptors

In [10]:
# A list of desriptors
Descriptors.descList

[('MaxEStateIndex',
  <function rdkit.Chem.EState.EState.MaxEStateIndex(mol, force=1)>),
 ('MinEStateIndex',
  <function rdkit.Chem.EState.EState.MinEStateIndex(mol, force=1)>),
 ('MaxAbsEStateIndex',
  <function rdkit.Chem.EState.EState.MaxAbsEStateIndex(mol, force=1)>),
 ('MinAbsEStateIndex',
  <function rdkit.Chem.EState.EState.MinAbsEStateIndex(mol, force=1)>),
 ('qed',
  <function rdkit.Chem.QED.qed(mol, w=QEDproperties(MW=0.66, ALOGP=0.46, HBA=0.05, HBD=0.61, PSA=0.06, ROTB=0.65, AROM=0.48, ALERTS=0.95), qedProperties=None)>),
 ('MolWt', <function rdkit.Chem.Descriptors.<lambda>(*x, **y)>),
 ('HeavyAtomMolWt', <function rdkit.Chem.Descriptors.HeavyAtomMolWt(x)>),
 ('ExactMolWt', <function rdkit.Chem.Descriptors.<lambda>(*x, **y)>),
 ('NumValenceElectrons',
  <function rdkit.Chem.Descriptors.NumValenceElectrons(mol)>),
 ('NumRadicalElectrons',
  <function rdkit.Chem.Descriptors.NumRadicalElectrons(mol)>),
 ('MaxPartialCharge',
  <function rdkit.Chem.Descriptors.MaxPartialCharge(mo

In [9]:
# Write a dictionary of name:function pairs for all descriptors
all_descriptors = {d[0]: d[1] for d in Descriptors.descList}

In [10]:
# Initialise a new pandas df
rdkit_descriptors = pd.DataFrame(data = {"smiles": np.array((smiles_list)) })
rdkit_descriptors

,smiles
0,C1Cc2cccc3cccc1c23
1,CC(O)=O
2,CC(C)=O
3,CC#N
4,CC(=O)c1ccccc1
...,...
347,CC(C)CC(C)(C)C
348,CC(C)C(C)C(C)C
349,Cc1ccccc1C
350,Cc1cccc(C)c1


In [11]:
# Compute each descriptor (outer loop) for each molecule(inside)
for feature in all_descriptors:
    values = []
    for mol in rdkit_mols:
        values += [all_descriptors[feature](mol)]
    rdkit_descriptors[feature] = values

rdkit_descriptors

[15:15:26] WARNING: not removing hydrogen atom without neighbors
[15:15:27] WARNING: not removing hydrogen atom without neighbors
/var/folders/dh/vv99823963l0gp7rpz76xwpw0000gn/T/ipykernel_12421/2319821685.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  rdkit_descriptors[feature] = values
/var/folders/dh/vv99823963l0gp7rpz76xwpw0000gn/T/ipykernel_12421/2319821685.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  rdkit_descriptors[feature] = values
/var/folders/dh/vv99823963l0gp7rpz76xwpw0000gn/T/ipykernel_12421/2319821685.

,smiles,MaxEStateIndex,MinEStateIndex,MaxAbsEStateIndex,MinAbsEStateIndex,qed,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,C1Cc2cccc3cccc1c23,2.248843,1.232824,2.248843,1.232824,0.546961,154.212,144.132,154.078250,58,...,0,0,0,0,0,0,0,0,0,0
1,CC(O)=O,9.000000,-0.833333,9.000000,0.833333,0.429883,60.052,56.020,60.021129,24,...,0,0,0,0,0,0,0,0,0,0
2,CC(C)=O,9.444444,0.166667,9.444444,0.166667,0.398237,58.080,52.032,58.041865,24,...,0,0,0,0,0,0,0,0,0,0
3,CC#N,7.319444,1.430556,7.319444,1.430556,0.386981,41.053,38.029,41.026549,16,...,0,0,0,0,0,0,0,0,0,0
4,CC(=O)c1ccccc1,10.645370,0.120926,10.645370,0.120926,0.517047,120.151,112.087,120.057515,46,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
347,CC(C)CC(C)(C)C,2.284722,0.521991,2.284722,0.521991,0.491154,114.232,96.088,114.140851,50,...,0,0,0,0,0,0,0,0,0,0
348,CC(C)C(C)C(C)C,2.314815,0.842593,2.314815,0.842593,0.517491,114.232,96.088,114.140851,50,...,0,0,0,0,0,0,0,0,0,0
349,Cc1ccccc1C,2.120370,1.368056,2.120370,1.368056,0.475758,106.168,96.088,106.078250,42,...,0,0,0,0,0,0,0,0,0,0
350,Cc1cccc(C)c1,2.166667,1.337963,2.166667,1.337963,0.475758,106.168,96.088,106.078250,42,...,0,0,0,0,0,0,0,0,0,0


In [26]:
rdkit_descriptors.to_csv("freesolv_rdkit_descriptors.csv")

In [14]:
# Finally, mordred descriptors
from mordred import Calculator, descriptors, error

In [16]:
# Initialise a calculator -- mordred works weirdly this way...
calc = Calculator(descriptors)

In [17]:
# Wow, many descriptors, much wow
len(calc.descriptors)

1826

In [18]:
mordred_descriptors = calc.pandas(rdkit_mols)

100%|████████████████████████████████████████████████████████████████████████████████| 642/642 [00:13<00:00, 47.62it/s]


In [19]:
# It seems that unfortunately some descriptors cannot be computed. To filter this, 
# we find all columns that are of data type "object", since those contain non-numerical values usually.
error_columns = []
for i, e in enumerate(mordred_descriptors.dtypes):
    if e=="object":
        error_columns += [i]
error_columns

[15,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 137,
 138,
 139,
 140,
 141,
 142,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 164,
 165,
 166,
 167,
 168,
 169,
 173,
 174,
 175,
 176,
 177,
 178,
 182,
 183,
 184,
 185,
 186,
 187,
 191,
 192,
 193,
 194,
 195,
 196,
 200,
 201,
 202,
 203,
 204,
 205,
 209,
 210,
 211,
 212,
 213,
 214,
 218,
 219,
 220,
 221,
 222,
 223,
 227,
 228,
 229,
 230,
 231,
 232,
 260,
 261,
 262,
 263,
 264,
 265,
 266,
 267,
 268,
 344,
 345,
 346,
 347,
 348,
 349,
 353,
 354,
 355,
 356,
 357,
 358,
 362,
 363,
 364,
 365,
 366,
 367,
 368,
 369,
 370,
 371,
 372,
 373,
 374,
 375,
 376,
 380,
 381,
 382,
 383,
 384,
 385,
 389,
 390,
 391,
 392,
 393,
 394,
 398,
 399,
 400,
 401,
 402,
 403,
 407,
 408,
 409,
 410,
 411,
 412,
 416,
 417,
 418,
 419,
 420,
 421,
 425,
 426,
 427,
 428,
 429,
 430,
 434,
 435,
 436,
 437,
 438,
 439,
 443,
 444,
 445,
 446,
 447,
 448,
 451,
 452,
 453,
 454,
 455,
 4

In [32]:
# use .drop to remove the affected columns 
mordred_descriptors = mordred_descriptors.drop(mordred_descriptors.columns[error_columns], axis=1)

In [33]:
# and remove columns containing NA data, but I don't think this actually does anything...
mordred_descriptors = mordred_descriptors.dropna(axis=1)

In [34]:
# again, insert first SMILES column
mordred_descriptors.insert(0, "smiles", smiles_list)
mordred_descriptors

,smiles,ABC,ABCGG,nAcid,nBase,SpAbs_A,SpMax_A,SpDiam_A,SpAD_A,SpMAD_A,...,SRW09,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb2
0,CN(C)C(=O)c1ccc(cc1)OC,9.439677,8.813176,0,0,16.051962,2.288246,4.576491,16.051962,1.234766,...,0.000000,9.147081,43.579108,179.094629,6.888255,260,17,60.0,67.0,3.055556
1,CS(=O)(=O)Cl,3.464102,3.464102,0,0,4.000000,2.000000,4.000000,4.000000,0.800000,...,0.000000,7.625107,29.418928,113.954228,14.244279,16,0,20.0,16.0,1.000000
2,CC(C)C=C,3.047207,3.305183,0,0,5.226252,1.847759,3.695518,5.226252,1.045250,...,0.000000,6.834109,27.254130,70.078250,4.671883,18,2,16.0,14.0,1.333333
3,CCc1cnccn1,5.656854,5.427660,0,0,10.424292,2.135779,4.271558,10.424292,1.303037,...,0.000000,8.298291,35.247635,108.068748,6.754297,64,7,34.0,36.0,2.000000
4,CCCCCCCO,4.949747,5.143137,0,0,9.517541,1.879385,3.758770,9.517541,1.189693,...,0.000000,7.126891,32.187603,116.120115,4.838338,84,5,26.0,24.0,2.250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
637,CCCCCCCC(=O)OC,7.180458,7.101974,0,0,13.045879,2.010756,4.021513,13.045879,1.185989,...,0.000000,7.937732,38.044586,158.130680,5.452782,206,9,40.0,39.0,2.916667
638,C1CCNC1,3.535534,3.535534,0,1,6.472136,2.000000,3.618034,6.472136,1.294427,...,5.888878,7.147559,41.004802,71.073499,5.076679,15,0,20.0,20.0,1.250000
639,c1cc(ccc1C=O)O,6.473351,6.127583,0,0,11.189957,2.193993,4.387987,11.189957,1.243329,...,0.000000,8.590258,37.289972,122.036779,8.135785,90,9,40.0,43.0,2.166667
640,CCCCCCCCl,4.949747,5.143137,0,0,9.517541,1.879385,3.758770,9.517541,1.189693,...,0.000000,7.126891,32.187603,134.086228,5.829836,84,5,26.0,24.0,2.250000


In [36]:
mordred_descriptors.to_csv("freesolv_mordred_descriptors.csv")

In [38]:
# finally, generate images of molecules
from rdkit.Chem import Draw
for i,mol in enumerate(rdkit_mols):
    Draw.MolToFile(mol, filename = "molecule_images/"+ str(i) + ".png")